# Bridges

From a national data set to local map...

In [24]:
import pandas as pd
import numpy as np
import folium


bridges = pd.read_csv("slubkin_992016-20170106140351.txt", encoding='cp1252', dtype={'LAT_016': object, 'LONG_017': object})
bridges['WSJ_YEARS_SINCE_BUILT'] = 2017 - bridges['YEAR_BUILT_027']
bridges.head()

/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,5,7,17,31,36,37,38,39,71,73,75,96,98,102,107,108,109,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,NBI_TYPE_OF_IMP,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE,CAT10,WSJ_YEARS_SINCE_BUILT
0,1,00000000000S702,1,6,1,04007,0,9,53.0,0,...,NaN,NaN,NaN,NaN,0,NaN,100.0,0,G,18
1,1,00000000000S703,1,6,1,04003,0,9,53.0,3004,...,NaN,NaN,NaN,NaN,0,NaN,100.0,0,G,15
2,1,0000000000M0022,1,8,1,00000,0,0,113.0,27568,...,NaN,NaN,NaN,NaN,0,NaN,69.5,0,F,75
3,1,000000883039900,1,4,1,00088,0,2,59.0,0,...,NaN,NaN,NaN,NaN,0,NaN,90.2,0,G,43
4,1,000001014002450,1,3,1,00101,0,2,79.0,0,...,NaN,NaN,NaN,NaN,2,NaN,49.9,2,F,80


In [25]:
bridges['WSJ_YEARS_SINCE_BUILT'].mean()

44.141267637498842

In [20]:
tmp = bridges[(~bridges['LAT_016'].isnull()) | (~bridges['LONG_017'].isnull())]
print(len(bridges))
print(len(tmp))
bridges = tmp

614387
614380


In [23]:
bridges[(bridges['STRUCTURE_NUMBER_008'] == '059B00046N     ')]['FEATURES_DESC_006A']

213587    'OHIO RIVER              '
Name: FEATURES_DESC_006A, dtype: object

In [2]:
print(len(bridges))
deficient_bridges = bridges[(bridges['STATUS_WITH_10YR_RULE'] == 1) | (bridges['STATUS_WITH_10YR_RULE'] == 2)]
print(len(deficient_bridges))

614387
131008


In [3]:
deficient_bridges.head().to_csv("def-bridges-head.csv")

In [4]:
def convert_latlng(frame):
    frame['lat_len'] = frame['LAT_016'].apply(lambda x: len(str(x)))
    frame['lat_degrees'] = frame['LAT_016'].apply(lambda x: float(str(x)[0:2]))
    frame['lat_minutes'] = frame['LAT_016'].apply(lambda x: float(str(x)[2:4]))
    frame['lat_seconds'] = frame['LAT_016'].apply(lambda x: float(str(x)[4:])/100)
    frame['lat'] = frame.apply(lambda x: x['lat_degrees'] + (x['lat_minutes']/60.0) + (x['lat_seconds']/3600.0), axis=1)

    frame['long_len'] = frame['LONG_017'].apply(lambda x: float(str(x)))
    frame['long_degrees'] = frame['LONG_017'].apply(lambda x: float(str(x)[0:3]))
    frame['long_minutes'] = frame['LONG_017'].apply(lambda x: float(str(x)[3:5]))
    frame['long_seconds'] = frame['LONG_017'].apply(lambda x: float(str(x)[5:])/100)
    frame['long'] = frame.apply(lambda x: x['long_degrees'] + (x['long_minutes']/60.0) + (x['long_seconds']/3600.0), axis=1)
    frame['long'] = frame['long'] * -1
    #frame.head()[['lat', 'long', 'LAT_016', 'LONG_017']]

#deficient_bridges.head()



In [5]:
convert_latlng(deficient_bridges)
deficient_bridges.to_csv("def-bridges.csv")
deficient_bridges.head()

/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,lat_len,lat_degrees,lat_minutes,lat_seconds,lat,long_len,long_degrees,long_minutes,long_seconds,long
4,1,000001014002450,1,3,1,00101,0,2,79.0,0,...,8,34.0,48.0,18.00,34.805000,87225400.0,87.0,22.0,54.00,-87.381667
5,1,000001331700710,1,6,2,00000,0,2,33.0,0,...,8,34.0,48.0,0.00,34.800000,87373000.0,87.0,37.0,30.00,-87.625000
6,1,000002,1,4,1,00000,0,10,127.0,0,...,8,33.0,54.0,30.30,33.908417,87171770.0,87.0,17.0,17.70,-87.288250
7,1,000004,1,4,1,00000,0,10,127.0,0,...,8,33.0,44.0,9.50,33.735972,87213460.0,87.0,21.0,34.60,-87.359611
9,1,000005,1,5,0,00000,0,10,75.0,48808,...,8,33.0,33.0,57.21,33.565892,88042578.0,88.0,4.0,25.78,-88.073828


In [22]:
dual_state = bridges[~bridges['OTHER_STATE_CODE_098A'].isnull()]
dual_state['WSJ_YEARS_SINCE_BUILT'] = 2017 - dual_state['YEAR_BUILT_027']
convert_latlng(dual_state)
print(len(dual_state))
dual_state.to_csv("dual-state.csv")

dual_state.head()

/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

1534


,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,WSJ_YEARS_SINCE_BUILT,lat_degrees,lat_minutes,lat_seconds,lat,long_len,long_degrees,long_minutes,long_seconds,long
4272,1,6628,1,3,1,00052,0,7,69.0,0,...,58,31.0,17.0,7.74,31.285483,85060296.0,85.0,6.0,2.96,-85.100822
5066,1,7641,1,2,1,00082,0,7,5.0,24568,...,55,31.0,53.0,29.58,31.891550,85080672.0,85.0,8.0,6.72,-85.135200
7970,1,11201,1,2,1,00084,0,7,69.0,0,...,43,31.0,7.0,25.23,31.123675,85032538.0,85.0,3.0,25.38,-85.057050
8936,1,12386,1,2,1,00098,0,9,3.0,0,...,37,30.0,24.0,28.59,30.407942,87261051.0,87.0,26.0,10.51,-87.436253
10284,1,13951,1,2,1,00098,0,9,97.0,0,...,31,30.0,51.0,48.82,30.863561,88250825.0,88.0,25.0,8.25,-88.418958


In [70]:
toshow = [
    'LAT_016',
    'lat',
    'lat_degrees',
    'lat_minutes',
    'lat_seconds',
    'LONG_017',
    'long',
    'long_degrees',
    'long_minutes',
    'long_seconds',
]
deficient_bridges[(deficient_bridges['STRUCTURE_NUMBER_008'] == '059B00046N     ')][toshow]

,LAT_016,lat,lat_degrees,lat_minutes,lat_seconds,LONG_017,long,long_degrees,long_minutes,long_seconds
213587,39052600,39.090556,39.0,5.0,26.0,084312100,-84.5225,84.0,31.0,21.0


## Bridge Inventory: https://www.fhwa.dot.gov/bridge/nbi.cfm
## Data Dictionary: https://www.fhwa.dot.gov/bridge/nbi/format.cfm
## Guide: https://www.fhwa.dot.gov/bridge/mtguide.pdf
## Data: https://www.fhwa.dot.gov/bridge/nbi/ascii.cfm

### degrees, minutes, seconds: http://www.augustatech.edu/math/molik/DegreesTransparency.pdf


### How many bridges are in New York City?

In [3]:
bx_fips = 5
bk_fips = 47
manhattan_fips = 61
queens_fips = 81
staten_fips = 85
nyc_county_fips = [
    bx_fips,
    bk_fips,
    manhattan_fips,
    queens_fips,
    staten_fips
]

buro_map = [
    'bronx',
    'brooklyn',
    'manhattan',
    'queens',
    'staten island'
]

nyc_bridges = bridges[bridges['COUNTY_CODE_003'].isin(nyc_county_fips)]
nyc_bridges['buro'] = nyc_bridges.apply(lambda x: buro_map[nyc_county_fips.index(x['COUNTY_CODE_003'])], axis=1)
print(len(nyc_bridges['STRUCTURE_NUMBER_008'].unique()))
len(nyc_bridges)

1442


/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1442

### How many 'structurally deficient' bridges are in NYC?

The worst bridges are generally those with a score below 50 percent, are classified as “structurally deficient” due to at least one defect that requires attention. OR

Status field...

1 = Structurally Deficient;
2 = Functionally Obsolete;
0 = Not Deficient;
N = Not Applicable

In [4]:
deficient_nyc_bridges = nyc_bridges[nyc_bridges['STATUS_NO_10YR_RULE'] == 1]
len(deficient_nyc_bridges)
#bridges that span two states OTHER_STATE_CODE_098A


141

### Rank buros by count of deficient bridges

In [5]:
deficient_by_buro = pd.pivot_table(deficient_nyc_bridges,index=["buro"], values=["STRUCTURE_NUMBER_008"], aggfunc=lambda x: len(x.unique()))
deficient_by_buro.reset_index().sort_values(by=['STRUCTURE_NUMBER_008'])

,buro,STRUCTURE_NUMBER_008
4,staten island,4
1,brooklyn,26
2,manhattan,28
3,queens,36
0,bronx,47


### What is the estimated cost to repair these bridges?

In [6]:
#TOTAL_IMP_COST_096
cost_by_buro = pd.pivot_table(deficient_nyc_bridges,index=["buro"], values=["TOTAL_IMP_COST_096"], aggfunc=np.sum)
cost_by_buro.reset_index().sort_values(by=['TOTAL_IMP_COST_096'])


,buro,TOTAL_IMP_COST_096
4,staten island,29183.0
1,brooklyn,1650353.0
3,queens,2056792.0
2,manhattan,2067870.0
0,bronx,2958936.0


In [7]:
cost_count_by_buro = pd.concat((deficient_by_buro, cost_by_buro), axis=1).reset_index()
cost_count_by_buro['TOTAL_IMP_COST_096'] = cost_count_by_buro['TOTAL_IMP_COST_096'].apply(lambda x: '${:,}'.format(x))
cost_count_by_buro

,buro,STRUCTURE_NUMBER_008,TOTAL_IMP_COST_096
0,bronx,47,"$2,958,936.0"
1,brooklyn,26,"$1,650,353.0"
2,manhattan,28,"$2,067,870.0"
3,queens,36,"$2,056,792.0"
4,staten island,4,"$29,183.0"


### Convert lat/lng


In [8]:
deficient_nyc_bridges['lat_degrees'] = deficient_nyc_bridges['LAT_016'].apply(lambda x: float(str(x/1000000).split('.')[0]))
deficient_nyc_bridges['lat_minutes'] = deficient_nyc_bridges['LAT_016'].apply(lambda x: float(str(x/1000000).split('.')[-1][0:2]))
deficient_nyc_bridges['lat_seconds'] = deficient_nyc_bridges['LAT_016'].apply(lambda x: float(str(x/1000000).split('.')[-1][2:4]))
deficient_nyc_bridges['lat'] = deficient_nyc_bridges.apply(lambda x: x['lat_degrees'] + (x['lat_minutes']/60.0) + (x['lat_seconds']/3600.0), axis=1)

deficient_nyc_bridges['long_degrees'] = deficient_nyc_bridges['LONG_017'].apply(lambda x: float(str(x/1000000).split('.')[0]))
deficient_nyc_bridges['long_minutes'] = deficient_nyc_bridges['LONG_017'].apply(lambda x: float(str(x/1000000).split('.')[-1][0:2]))
deficient_nyc_bridges['long_seconds'] = deficient_nyc_bridges['LONG_017'].apply(lambda x: float(str(x/1000000).split('.')[-1][2:4]))
deficient_nyc_bridges['long'] = deficient_nyc_bridges.apply(lambda x: x['long_degrees'] + (x['long_minutes']/60.0) + (x['long_seconds']/3600.0), axis=1)
deficient_nyc_bridges['long'] = deficient_nyc_bridges['long'] * -1
deficient_nyc_bridges.head()[['lat', 'long', 'LAT_016', 'LONG_017']]

/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ShaneShifflett/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,lat,long,LAT_016,LONG_017
4386,40.740556,-73.786667,40442699,73471243
4391,40.743889,-73.771389,40443870,73461764
4397,40.747222,-73.761111,40445070,73454053
4418,40.674167,-73.801389,40402750,73480559
4436,40.762500,-73.839722,40454509,73502363


In [12]:
deficient_nyc_bridges[deficient_nyc_bridges['LAT_016'] == 40435073]

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,CAT10,buro,lat_degrees,lat_minutes,lat_seconds,lat,long_degrees,long_minutes,long_seconds,long


### Maps!

https://rawgit.com/CoulterJones/New-School/master/news-narrative-design/spring-2017/level-two/bridges.html

http://rawgit.com/

http://www.convertcsv.com/csv-to-geojson.htm

In [ ]:
bridge_map = folium.Map(location=[40.7128, -74.0059], tiles='Stamen Toner',
                    zoom_start=13)

for row in deficient_nyc_bridges.iterrows():
    idx = row[0]
    obj = row[1]
    latlng = [obj['lat'], obj['long']]
    rating = obj['SUFFICIENCY_RATING']
    marker = folium.CircleMarker(location=latlng, radius=100,
                    color='red',
                    fill_color='red')
    marker.add_to(bridge_map)
bridge_map.save('bridges.html')

In [ ]:
deficient_nyc_bridges[['lat', 'long']].to_csv("nyc-deficient-bridges-locs.csv")